In [1]:
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import json
import pandas as pd

In [2]:
#connecting to Spotify API

cid ="0cc19d9dac2345bfad84755de43ff06f" 
secret = "b4d778a84ad44234849d0543ffc0c827"

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret) 
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [3]:
#URL to Taylor Swift's Spotify page
taylor_url = "https://open.spotify.com/artist/06HL4z0CvFAxyc27GXpf02?si=ZwUBcAj-RM-UXGcOEXzJ6A"

In [4]:
#API call pulling Taylor's albums
albums = sp.artist_albums(artist_id=taylor_url, album_type='album', country='US', limit=50, offset=0)

In [5]:
#loading album data into readable json
with open('tayloralbums_data.json', 'w', encoding='utf-8') as f:
            json.dump(albums, f, ensure_ascii=False, indent=4)
with open('tayloralbums_data.json') as f:
  album_data = json.load(f)

json_df = pd.read_json('tayloralbums_data.json')

In [7]:
#'unpacking' items column, selecting relevant versions of albums, and reseting the index
#had to select albums manually because there are multiple versions, singles, EPs, etc and I wanted only full length albums
#that were the deluxe versions and taylor's versions when applicable
album_df = json_df['items'].apply(pd.Series).iloc[[1,2,4,10,13,15,20,29,35],:].reset_index()

In [8]:
#creating a dict oof the album names and their uri's (Spotify identifier)
albums = dict(zip(album_df.name,album_df.uri))

In [9]:
#function to make an API call pulling the tracks for an album and loading themm into a dataframe
def get_tracks_from_album(album_name):
    album = sp.album_tracks(albums[album_name])
    with open('album.json', 'w', encoding='utf-8') as f:
        json.dump(album, f, ensure_ascii=False, indent=4)
    with open('album.json') as f:
        track_data = json.load(f)
    album = pd.read_json('album.json')
    tracks = album['items'].apply(pd.Series)
    tracks['album'] = album_name
    return tracks

In [10]:
#calling get_tracks function on each album and appending the results to a dataframe
all_album_tracks = []
for items in albums:
    all_album_tracks.append(get_tracks_from_album(items))
tracks = pd.concat(all_album_tracks[0:9])
tracks.reset_index(inplace=True)

In [11]:
#API call to get the audio features for each track, then loaded into a dataframe
audio_features = []
for i in range(0,len(tracks)):
    audio_features.append(sp.audio_features(tracks.loc[i,'uri']))
audio_features = pd.DataFrame(audio_features)

In [12]:
#'unpacking' audio_features dataframe and dropping unnecessary columns
dffeatures = audio_features.apply(pd.Series)[0].apply(pd.Series)
dffeatures.drop(['id','uri','track_href','type','time_signature', 'duration_ms'],axis=1,inplace=True)

In [13]:
#joining tracks dataframe to audio features dataframe, dropping unnecessary columns
#since order was preserved, used a simple merge
df = pd.concat([tracks,dffeatures],axis=1)
df.drop(["artists", 'available_markets','disc_number','href', 'is_local', 'preview_url', 'type','index', 'id'], axis=1, inplace=True)

In [29]:
#preview df then write it to csv
df.head()
df.to_csv('taylor_discography.csv', index=False)

,duration_ms,explicit,external_urls,name,track_number,uri,album,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,analysis_url
0,295413,False,{'spotify': 'https://open.spotify.com/track/5l...,State Of Grace (Taylor's Version),1,spotify:track:5lFDIbYMcFHPPgzMcnPqSz,Red (Taylor's Version),0.594,0.713,9,-5.314,1,0.0503,0.000328,0.000000,0.1140,0.328,129.958,https://api.spotify.com/v1/audio-analysis/5lFD...
1,223093,False,{'spotify': 'https://open.spotify.com/track/78...,Red (Taylor's Version),2,spotify:track:78c485atQVK19038U2RUld,Red (Taylor's Version),0.516,0.777,1,-4.908,0,0.0375,0.001080,0.000002,0.0761,0.408,125.047,https://api.spotify.com/v1/audio-analysis/78c4...
2,242720,False,{'spotify': 'https://open.spotify.com/track/0Y...,Treacherous (Taylor's Version),3,spotify:track:0YrMTMLpLq1UIkWgghNM6c,Red (Taylor's Version),0.645,0.593,2,-6.506,1,0.0288,0.034400,0.000127,0.1300,0.299,109.984,https://api.spotify.com/v1/audio-analysis/0YrM...
3,219760,False,{'spotify': 'https://open.spotify.com/track/6t...,I Knew You Were Trouble (Taylor's Version),4,spotify:track:6tNSmBKSjFV1366ASuULfZ,Red (Taylor's Version),0.584,0.557,6,-6.371,1,0.0342,0.012900,0.000000,0.0576,0.767,154.008,https://api.spotify.com/v1/audio-analysis/6tNS...
4,329160,False,{'spotify': 'https://open.spotify.com/track/1I...,All Too Well (Taylor's Version),5,spotify:track:1IBhYEvvi02tDVByxIiTh4,Red (Taylor's Version),0.440,0.528,0,-7.809,1,0.0317,0.017100,0.002030,0.2340,0.132,185.972,https://api.spotify.com/v1/audio-analysis/1IBh...
